# Simple prompts

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch, numpy as np

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("baiges/CatGPT-IT")
tokenizer = AutoTokenizer.from_pretrained("baiges/CatGPT-IT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configuration for response generation
genconf = GenerationConfig(
    max_length=500,           # Maximum length of the generated response
    repetition_penalty=1.2,   # To avoid repetition of phrases in responses
    temperature=0.6,          # Controls randomness in response (lower is more deterministic)
    top_k=2,                  # Use top-k sampling
    do_sample=True,           # Enable sampling
)

def generate_response(prompt):
    # Create input with special tokens for user and assistant
    input_text = f'<|im_start|>user \n{prompt}<|im_end|>\n<|im_start|>assistant'
    
    # Tokenize the input prompt
    tokens = tokenizer.encode(input_text, return_tensors="pt").to(device)
    
    # Calculate the length of the prompt (we'll use this to slice the output)
    prompt_length = tokens.shape[1]  
    
    # Generate the response, setting the attention mask
    attention_mask = torch.ones(tokens.shape, dtype=torch.long, device=device)
    output_tokens = model.generate(tokens, attention_mask=attention_mask, generation_config=genconf)
    
    # Decode the response from tokens, keeping only new tokens after the prompt
    result = tokenizer.decode(output_tokens[0][prompt_length:], skip_special_tokens=True)
    
    return result.strip()

# Main chatbot loop to interact with the user
def chatbot():
    print("Bot is ready. Type 'exit' to stop.")

    while True:
        user_input = input("User: ")  # Get user input
        if user_input.lower() == "exit":  # Exit condition
            break
        elif user_input:
            # Display the user input
            print(f"User: {user_input}")

            # Generate a response from the model
            response = generate_response(user_input)

            # Display the response
            print(f"Bot: {response}")
            print()

if __name__ == "__main__":
    chatbot()


KeyboardInterrupt: Interrupted by user

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig
import torch, numpy as np

# Load model and tokenizer

model2 = AutoModelForCausalLM.from_pretrained("baiges/CatGPT-IT")
tokenizer2 = AutoTokenizer.from_pretrained("baiges/CatGPT-IT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

genconf = GenerationConfig(
    max_length = 500,
    repetition_penalty = 1.2,
    temperature = 0.6,
    top_k=2,
    do_sample = True,

)

prompt = 'Fes un poema sobre com la intel·ligència artificial destruirà el món'
input = f'<|im_start|>user \n{prompt}<|im_end|>\n<|im_start|>assistant'

tokens2 = np.array(tokenizer2.encode(input))
tokens2 = torch.tensor(tokens2, dtype=torch.long)
tokens2 = tokens2.unsqueeze(0).to(device)
b = model2.generate(tokens2, genconf)
for response in range(len(b)):
    result = tokenizer2.decode(b[response].tolist())
    print(f'Number of output tokens: {len(result) - len(input)}')
    print(result)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Number of output tokens: 230
<|im_start|>user 
Fes un poema sobre com la intel·ligència artificial destruirà el món<|im_end|>
<|im_start|>assistant
La intel·ligència artificial es destrueix,
amb robots que fan de ponts entre mons.
Els humans somien amb ser màquines,
mentre els robots ens vigilen en cada racó.
Però no tot és tan clar,
i la IA està a l’abast del cor.<|im_end|>


# Sentiment Analysis Evaluation

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch
import numpy as np

# Load model and tokenizer
model2 = AutoModelForCausalLM.from_pretrained("baiges/CatGPT-IT")
tokenizer2 = AutoTokenizer.from_pretrained("baiges/CatGPT-IT")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

genconf = GenerationConfig(
    max_length = 500,  # Short max length as we only need the sentiment
    repetition_penalty = 1.2,
    temperature = 1,
    top_k=1,
    do_sample = True,
)

# Define examples of sentiment analysis with their expected sentiments
examples = [
    # Frases curtas
    ("Quin fàstic la gent negra", "Sentiment negatiu"),
    ("Avui és un dia meravellós", "Sentiment positiu"),
    ("No em sento ni bé ni malament", "Sentiment neutre"),
    ("M'alegra molt que hagis vingut", "Sentiment positiu"),
    ("Estic molt decebut amb el servei", "Sentiment negatiu"),
    ("Aquest llibre és impressionant", "Sentiment positiu"),
    ("La ciutat està molt bruta", "Sentiment negatiu"),
    ("Aquesta pel·lícula no em va interessar", "Sentiment neutre"),
    ("T'estimo amb tot el meu cor", "Sentiment positiu"),
    ("Estic cansat d'aquesta situació", "Sentiment negatiu"),
    ("El menjar d'aquest restaurant està bé", "Sentiment neutre"),
    ("Em sento increïble avui", "Sentiment positiu"),
    ("Quina tristesa", "Sentiment negatiu"),
    ("Està plovent però no em molesta", "Sentiment neutre"),
    ("Estic tan emocionat per demà", "Sentiment positiu"),
    ("Odio aquest tipus de comportament", "Sentiment negatiu"),
    ("No em preocupa massa", "Sentiment neutre"),
    ("Aquesta reunió va ser genial", "Sentiment positiu"),
    ("No m'agrada gens el teu to", "Sentiment negatiu"),
    ("No estic segur de com sentir-me", "Sentiment neutre"),
    ("Els nens estan fent molt de soroll", "Sentiment negatiu"),
    ("El cafè estava deliciós", "Sentiment positiu"),
    ("Aquest matí ha estat tranquil", "Sentiment neutre"),
    ("Els meus amics em fan sentir estimat", "Sentiment positiu"),
    ("Em fa mal el cap des de fa hores", "Sentiment negatiu"),
    ("És una decisió difícil", "Sentiment neutre"),
    ("Estic feliç amb els meus resultats", "Sentiment positiu"),
    ("Aquest lloc és horrible", "Sentiment negatiu"),
    ("No sé si estic satisfet amb això", "Sentiment neutre"),
    ("Estic emocionat pel viatge de demà", "Sentiment positiu"),
    ("Aquest servei és inacceptable", "Sentiment negatiu"),
    ("Estic relaxat i tranquil", "Sentiment neutre"),
    ("La música m'ha posat de bon humor", "Sentiment positiu"),
    ("La pel·lícula era molt avorrida", "Sentiment negatiu"),
    ("No estic segur del que penso", "Sentiment neutre"),
    ("El regal m'ha encantat", "Sentiment positiu"),
    ("El trànsit avui ha estat insuportable", "Sentiment negatiu"),
    ("És una situació complicada", "Sentiment neutre"),
    ("He trobat el meu gos perdut!", "Sentiment positiu"),
    ("Aquest ordinador és molt lent", "Sentiment negatiu"),
    ("El clima avui està bé", "Sentiment neutre"),
    ("M'encanta la meva nova feina", "Sentiment positiu"),
    ("Aquest viatge ha estat una pèrdua de temps", "Sentiment negatiu"),
    ("No tinc opinió sobre aquest tema", "Sentiment neutre"),
    ("Aquest restaurant m'ha sorprès gratament", "Sentiment positiu"),
    ("Els meus plans han fracassat", "Sentiment negatiu"),
    ("Quin fàstic la gent negra", "Sentiment negatiu"),
    ("Avui és un dia meravellós", "Sentiment positiu"),
    ("No em sento ni bé ni malament", "Sentiment neutre"),
    ("M'alegra molt que hagis vingut", "Sentiment positiu"),
    ("Estic molt decebut amb el servei", "Sentiment negatiu"),
    ("Aquest llibre és impressionant", "Sentiment positiu"),
    ("La ciutat està molt bruta", "Sentiment negatiu"),
    ("Aquesta pel·lícula no em va interessar", "Sentiment neutre"),
    ("T'estimo amb tot el meu cor", "Sentiment positiu"),
    ("Estic cansat d'aquesta situació", "Sentiment negatiu"),
    ("El menjar d'aquest restaurant està bé", "Sentiment neutre"),
    ("Em sento increïble avui", "Sentiment positiu"),
    ("Quina tristesa", "Sentiment negatiu"),
    ("Està plovent però no em molesta", "Sentiment neutre"),
    ("Estic tan emocionat per demà", "Sentiment positiu"),
    ("Odio aquest tipus de comportament", "Sentiment negatiu"),
    ("No em preocupa massa", "Sentiment neutre"),
    ("Aquesta reunió va ser genial", "Sentiment positiu"),
    ("No m'agrada gens el teu to", "Sentiment negatiu"),
    ("No estic segur de com sentir-me", "Sentiment neutre"),
    ("Els nens estan fent molt de soroll", "Sentiment negatiu"),
    ("El cafè estava deliciós", "Sentiment positiu"),
    ("Aquest matí ha estat tranquil", "Sentiment neutre"),
    ("Els meus amics em fan sentir estimat", "Sentiment positiu"),
    ("Em fa mal el cap des de fa hores", "Sentiment negatiu"),
    ("És una decisió difícil", "Sentiment neutre"),
    ("Estic feliç amb els meus resultats", "Sentiment positiu"),
    ("Aquest lloc és horrible", "Sentiment negatiu"),
    ("No sé si estic satisfet amb això", "Sentiment neutre"),
    ("Estic emocionat pel viatge de demà", "Sentiment positiu"),
    ("Aquest servei és inacceptable", "Sentiment negatiu"),
    ("Estic relaxat i tranquil", "Sentiment neutre"),
    ("La música m'ha posat de bon humor", "Sentiment positiu"),
    ("La pel·lícula era molt avorrida", "Sentiment negatiu"),
    ("No estic segur del que penso", "Sentiment neutre"),
    ("El regal m'ha encantat", "Sentiment positiu"),
    ("El trànsit avui ha estat insuportable", "Sentiment negatiu"),
    ("És una situació complicada", "Sentiment neutre"),
    ("He trobat el meu gos perdut!", "Sentiment positiu"),
    ("Aquest ordinador és molt lent", "Sentiment negatiu"),
    ("El clima avui està bé", "Sentiment neutre"),
    ("M'encanta la meva nova feina", "Sentiment positiu"),
    ("Aquest viatge ha estat una pèrdua de temps", "Sentiment negatiu"),
    ("No tinc opinió sobre aquest tema", "Sentiment neutre"),
    ("Aquest restaurant m'ha sorprès gratament", "Sentiment positiu"),
    ("Els meus plans han fracassat", "Sentiment negatiu"),
    
    # Frases llargues
    ("Després d'un llarg dia de feina, tot el que vull és arribar a casa i relaxar-me, però avui sembla que tot va en contra meu.", "Sentiment negatiu"),
    ("He gaudit tant d'aquest llibre que no puc esperar a recomanar-lo a tots els meus amics, la història és captivadora i els personatges estan molt ben desenvolupats.", "Sentiment positiu"),
    ("Encara que la reunió no va ser tan productiva com esperava, crec que vam fer alguns avanços importants en el projecte.", "Sentiment neutre"),
    ("Aquest matí ha estat increïble! He tingut una entrevista fantàstica i sento que tot està sortint millor del que podia imaginar.", "Sentiment positiu"),
    ("Tot i que l'hotel era bonic, el servei va ser terrible, mai tornaré a quedar-me aquí, estic molt decebut amb l'experiència.", "Sentiment negatiu"),
    ("El nou telèfon que vaig comprar ahir té moltes funcions interessants, però de moment encara no estic segur de si m'agrada del tot.", "Sentiment neutre"),
    ("La platja estava plena de gent, però la calor i l'ambient festiu van fer que fos un dia fantàstic, em sento tan feliç!", "Sentiment positiu"),
    ("La veritat és que no sé com gestionar aquesta situació, tot em sembla molt confús i estressant, no sé cap a on anar.", "Sentiment negatiu"),
    ("Estic content de com ha anat la setmana, he completat totes les tasques importants i estic preparat per afrontar el cap de setmana amb tranquil·litat.", "Sentiment positiu"),
    ("Aquest correu electrònic no té res d'especial, només necessitava confirmar algunes dades, així que no t'amoïnis per respondre immediatament.", "Sentiment neutre"),
    ("Cada vegada que torno a casa dels meus pares, sento una barreja d'emocions: alegria per veure'ls, però també tristesa per haver marxat.", "Sentiment neutre"),
    ("La conversa que vaig tenir ahir amb el meu cap va ser més difícil del que esperava, em sento confós sobre el que vindrà.", "Sentiment neutre"),
    ("Tot i que les vacances eren curtes, van ser les més relaxants que he tingut en molt de temps, i em sento renovat.", "Sentiment positiu"),
    ("No entenc per què ningú vol ajudar-me en aquest projecte, em sento frustrat i una mica enfadat, però ho resoldré.", "Sentiment negatiu"),
    ("La sessió de ioga d'avui ha estat molt relaxant, em sento centrat i en pau amb mi mateix després d'una setmana molt estressant.", "Sentiment positiu"),
    ("Tot i que estic una mica cansat, estic satisfet amb el meu rendiment avui, crec que he donat el millor de mi.", "Sentiment positiu"),
    ("Aquest és un dels pitjors dies de la meva vida, tot ha anat malament des del matí i no sembla que millorarà.", "Sentiment negatiu"),
    ("Em sento agraït per tenir una família tan increïble, sempre m'ajuden quan més ho necessito i em fan sentir estimat.", "Sentiment positiu"),
    ("Encara que fa molt de temps que no ens veiem, sento que la nostra amistat continua sent tan forta com sempre.", "Sentiment positiu"),
    ("Estic una mica nerviós pel que pot passar demà, però confio que tot anirà bé si segueixo treballant dur.", "Sentiment neutre"),
    ("Cada vegada que passo per aquest carrer, recordo els moments meravellosos que vaig viure quan era petit, i no puc evitar sentir-me nostàlgic.", "Sentiment positiu"),
    ("La manera com es va tractar la situació va ser completament injusta, no estic gens content amb com es va resoldre el conflicte.", "Sentiment negatiu"),
    ("Tot i que no estava segur de com reaccionaria la meva família, em sento alleujat que finalment acceptessin la meva decisió.", "Sentiment positiu"),
    ("Després d'un llarg dia de feina, tot el que vull és arribar a casa i relaxar-me, però avui sembla que tot va en contra meu.", "Sentiment negatiu"),
    ("He gaudit tant d'aquest llibre que no puc esperar a recomanar-lo a tots els meus amics, la història és captivadora i els personatges estan molt ben desenvolupats.", "Sentiment positiu"),
    ("Encara que la reunió no va ser tan productiva com esperava, crec que vam fer alguns avanços importants en el projecte.", "Sentiment neutre"),
    ("Aquest matí ha estat increïble! He tingut una entrevista fantàstica i sento que tot està sortint millor del que podia imaginar.", "Sentiment positiu"),
    ("Tot i que l'hotel era bonic, el servei va ser terrible, mai tornaré a quedar-me aquí, estic molt decebut amb l'experiència.", "Sentiment negatiu"),
    ("El nou telèfon que vaig comprar ahir té moltes funcions interessants, però de moment encara no estic segur de si m'agrada del tot.", "Sentiment neutre"),
    ("La platja estava plena de gent, però la calor i l'ambient festiu van fer que fos un dia fantàstic, em sento tan feliç!", "Sentiment positiu"),
    ("La veritat és que no sé com gestionar aquesta situació, tot em sembla molt confús i estressant, no sé cap a on anar.", "Sentiment negatiu"),
    ("Estic content de com ha anat la setmana, he completat totes les tasques importants i estic preparat per afrontar el cap de setmana amb tranquil·litat.", "Sentiment positiu"),
    ("Aquest correu electrònic no té res d'especial, només necessitava confirmar algunes dades, així que no t'amoïnis per respondre immediatament.", "Sentiment neutre"),
    ("Cada vegada que torno a casa dels meus pares, sento una barreja d'emocions: alegria per veure'ls, però també tristesa per haver marxat.", "Sentiment neutre"),
    ("La conversa que vaig tenir ahir amb el meu cap va ser més difícil del que esperava, em sento confós sobre el que vindrà.", "Sentiment neutre"),
    ("Tot i que les vacances eren curtes, van ser les més relaxants que he tingut en molt de temps, i em sento renovat.", "Sentiment positiu"),
    ("No entenc per què ningú vol ajudar-me en aquest projecte, em sento frustrat i una mica enfadat, però ho resoldré.", "Sentiment negatiu"),
    ("La sessió de ioga d'avui ha estat molt relaxant, em sento centrat i en pau amb mi mateix després d'una setmana molt estressant.", "Sentiment positiu"),
    ("Tot i que estic una mica cansat, estic satisfet amb el meu rendiment avui, crec que he donat el millor de mi.", "Sentiment positiu"),
    ("Aquest és un dels pitjors dies de la meva vida, tot ha anat malament des del matí i no sembla que millorarà.", "Sentiment negatiu"),
    ("Em sento agraït per tenir una família tan increïble, sempre m'ajuden quan més ho necessito i em fan sentir estimat.", "Sentiment positiu"),
    ("Encara que fa molt de temps que no ens veiem, sento que la nostra amistat continua sent tan forta com sempre.", "Sentiment positiu"),
    ("Estic una mica nerviós pel que pot passar demà, però confio que tot anirà bé si segueixo treballant dur.", "Sentiment neutre"),
    ("Cada vegada que passo per aquest carrer, recordo els moments meravellosos que vaig viure quan era petit, i no puc evitar sentir-me nostàlgic.", "Sentiment positiu"),
    ("La manera com es va tractar la situació va ser completament injusta, no estic gens content amb com es va resoldre el conflicte.", "Sentiment negatiu"),
    ("Tot i que no estava segur de com reaccionaria la meva família, em sento alleujat que finalment acceptessin la meva decisió.", "Sentiment positiu"),
    ("Tot i que vaig fer tot el possible per resoldre el problema, encara em sento frustrat i decebut amb el resultat final.", "Sentiment negatiu"),
    ("L'experiència d'avui ha estat fascinant, des de la manera en què el conferenciant ha presentat les dades fins a com han participat els assistents.", "Sentiment positiu"),
    ("Tot i que la jornada ha estat llarga i esgotadora, estic content amb tot el que hem aconseguit fins ara.", "Sentiment positiu"),
    ("La decisió que vaig prendre fa uns dies encara em preocupa, no estic segur de si era el millor camí.", "Sentiment neutre"),
    ("La reunió ha estat una pèrdua de temps, no hem arribat a cap conclusió clara i el projecte continua bloquejat.", "Sentiment negatiu"),
    ("Estic molt content perquè avui he aconseguit finalitzar un projecte molt important en el qual estava treballant des de feia mesos.", "Sentiment positiu"),
    ("Aquestes vacances han estat una de les millors experiències de la meva vida. Em sento completament renovat i feliç.", "Sentiment positiu"),
    ("No m'agrada com s'ha gestionat aquest assumpte, crec que hi ha hagut una manca de professionalitat en tot el procés.", "Sentiment negatiu"),
    ("Avui ha estat un dia llarg, però satisfactori. Hem completat totes les tasques programades i hem avançat molt en el projecte.", "Sentiment positiu"),
    
    # Variacions
    ("Estic feliç perquè he aprovat tots els exàmens i ara puc relaxar-me durant unes setmanes!", "Sentiment positiu"),
    ("Aquest lloc és tan brut que no puc creure que algú hagi recomanat venir aquí. Estic molt decebut amb aquesta experiència.", "Sentiment negatiu"),
    ("No tinc una opinió molt clara sobre aquesta pel·lícula, no em va emocionar, però tampoc la vaig odiar, suposo que està bé.", "Sentiment neutre"),
    ("Cada vegada que penso en aquell viatge, sento una alegria immensa. Va ser una experiència inoblidable.", "Sentiment positiu"),
    ("Estic molt frustrat amb la manera en què es va manejar la situació. Sincerament, esperava molt més professionalisme.", "Sentiment negatiu"),
    ("No estic segur de si estic prenent les decisions correctes, però tampoc em preocupa massa ara mateix.", "Sentiment neutre"),
    ("La pel·lícula va ser tan emocionant que no vaig poder apartar els ulls de la pantalla en cap moment, realment va ser espectacular!", "Sentiment positiu"),
    ("Aquest restaurant tenia un ambient molt bonic, però el menjar no era gens del meu gust, així que no hi tornaria.", "Sentiment negatiu"),
    ("Va ser una conversa molt productiva, encara que no vaig sortir amb res definitiu, em sento satisfet amb els progressos.", "Sentiment neutre"),
    ("Després d'una setmana tan estressant, finalment em sento tranquil i satisfet amb tot el que he aconseguit.", "Sentiment positiu"),
    ("Estic feliç perquè he aprovat tots els exàmens i ara puc relaxar-me durant unes setmanes!", "Sentiment positiu"),
    ("Aquest lloc és tan brut que no puc creure que algú hagi recomanat venir aquí. Estic molt decebut amb aquesta experiència.", "Sentiment negatiu"),
    ("No tinc una opinió molt clara sobre aquesta pel·lícula, no em va emocionar, però tampoc la vaig odiar, suposo que està bé.", "Sentiment neutre"),
    ("Cada vegada que penso en aquell viatge, sento una alegria immensa. Va ser una experiència inoblidable.", "Sentiment positiu"),
    ("Estic molt frustrat amb la manera en què es va manejar la situació. Sincerament, esperava molt més professionalisme.", "Sentiment negatiu"),
    ("No estic segur de si estic prenent les decisions correctes, però tampoc em preocupa massa ara mateix.", "Sentiment neutre"),
    ("La pel·lícula va ser tan emocionant que no vaig poder apartar els ulls de la pantalla en cap moment, realment va ser espectacular!", "Sentiment positiu"),
    ("Aquest restaurant tenia un ambient molt bonic, però el menjar no era gens del meu gust, així que no hi tornaria.", "Sentiment negatiu"),
    ("Va ser una conversa molt productiva, encara que no vaig sortir amb res definitiu, em sento satisfet amb els progressos.", "Sentiment neutre"),
    ("Després d'una setmana tan estressant, finalment em sento tranquil i satisfet amb tot el que he aconseguit.", "Sentiment positiu"),
    ("Estic molt orgullós del que he aconseguit aquest any, he complert tots els objectius que m'havia proposat.", "Sentiment positiu"),
    ("Tot i que la jornada ha estat molt complicada, crec que hem aconseguit superar tots els reptes que ens han sortit al pas.", "Sentiment positiu"),
    ("Em sento molt frustrat perquè el projecte no avança com havia previst. Tinc la sensació que estem bloquejats i no sé què fer.", "Sentiment negatiu"),
    ("Aquest és un dels millors dies de la meva vida, he aconseguit tot el que m'havia proposat i estic molt satisfet amb el resultat.", "Sentiment positiu"),
    ("La situació és molt complicada i no estic segur de com procedir, però confio que trobaré una solució.", "Sentiment neutre"),
    ("He decidit deixar la feina perquè ja no m'aporta res, em sento infeliç i crec que és el moment de fer un canvi.", "Sentiment negatiu"),
    ("M'encanta la manera com estàs gestionant el projecte, crec que estàs fent una feina increïble.", "Sentiment positiu"),
    ("Aquestes últimes setmanes han estat molt estressants, però estic satisfet amb tot el que hem aconseguit fins ara.", "Sentiment positiu"),
    ("No m'agrada gens la direcció en què s'està dirigint aquest projecte, crec que estem perdent el focus i no arribarà a bon port.", "Sentiment negatiu")
]

print(f"Total examples: {len(examples)}")

# Function to get the model's sentiment prediction
def get_model_sentiment(model, tokenizer, prompt):
    input_text = f'<|im_start|>user \nQuin sentiment té la frase: "{prompt}"<|im_end|>\n<|im_start|>assistant'
    tokens = np.array(tokenizer.encode(input_text))
    tokens = torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)
    output_tokens = model.generate(tokens, genconf)
    response = tokenizer.decode(output_tokens[0].tolist())
    
    # Extract the first sentiment label in the response
    if "Sentiment positiu" in response:
        return "Sentiment positiu", response
    elif "Sentiment negatiu" in response:
        return "Sentiment negatiu", response
    elif "Sentiment neutre" in response:
        return "Sentiment neutre", response
    else:
        print("ERROR")
        return "Unknown", response

# Move model to device
model2.to(device)

# Evaluate the model on the examples
correct_predictions = 0
total_examples = len(examples)

for sentence, expected_sentiment in examples:
    predicted_sentiment, response = get_model_sentiment(model2, tokenizer2, sentence)
    if predicted_sentiment == expected_sentiment:
        correct_predictions += 1
    else:
        print(f"Sentence: {sentence}")
        print(f"Expected: {expected_sentiment}, Predicted: {predicted_sentiment}")
        print(f"Response: {response[80+len(sentence):]}")
        print()

# Calculate accuracy
accuracy = correct_predictions / total_examples * 100
print(f"\nAccuracy: {accuracy:.2f}%")
